In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.0     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
position_file <- NA #"/mnt/archgen/public_data/HGDP+1KG_callset/stats/POS.all_filtered.AFR_mono.5EUR/PS.HGDP+1000G.EUR_asc_min258.txt"

frequency_files_list <- "example/frequency_files_list.txt"
# allowed non-frequency columns: CHR; POS; ALT; REF

pop_def_file <- "example/pop_def.txt"

n_pops <- 3

popA_str <- "AFR_all"
# (1) directory to a text file, in which each row consists of one population
# (2) string separated by comma ","

popA_fillna <- NA # if it's an outgroup, it can only support NA
# (1) NA
# (2) HomoRef
# (3) Outgroup

popB_str <- "CEU,FIN,GBR,IBS,TSI"

popB_fillna <- "Outgroup"

popC_str <- "example/popC.txt"

popC_fillna <- NA

popD_str <- NA

popD_fillna <- NA

jackknife <- "CHR"

asc_refpop <- "EUR5"
# Whether it's ascertained depends on ascertainment file

refpop_max_miss <- 0.75

refpop_remove_homo <- TRUE

asc_outgroup <- "AFR_all"
# Outgroup only applies when ascertainment is defined.

outgroup_max_hetero <- 0

outgroup_round_freq <- FALSE
# use the rounded frequency (0/1) when outgroup is involved in F-stats


In [3]:
get_frequency_file_info <- function(frequency_file, header_info = NA)
{
    is_geno <- tools::file_ext(frequency_file) == "geno"
    if (is.na(header_info))
        header_info <- paste0(tools::file_path_sans_ext(frequency_file), if (is_geno) ".ind" else ".header")
    
    header <- ( if (file.exists(header_info)) utils::read.table(header_info, header = FALSE, sep = "\t")[[1]] else base::strsplit(header_info, ",")[[1]] ) %>% base::strsplit(split="[()]")
    return(data.frame(frequency_file = frequency_file, field = 1:base::length(header), column_name = sapply(header, `[`, 1), denominator = if (is_geno) 2 else as.numeric(sapply(header, `[`, 2))))
}

In [4]:
if (!file.exists(frequency_files_list))
{
    message("Frequency files list", frequency_files_list, "does not exist.")
    stop()
}
frequency_files <- utils::read.table(frequency_files_list, header = FALSE, sep = "\t", col.names = c("frequency_file", "header_info"), fill = TRUE, na.strings = c("NA", ""))
frequency_files_info <- purrr::pmap(frequency_files, get_frequency_file_info) %>% do.call(base::rbind, .)

column_names_duplicated <- frequency_files_info$column_name %>% .[base::duplicated(.)] %>% base::unique()
if (base::length(column_names_duplicated))
{
    message("The following column(s) in frequency file(s) is/are duplicated: ", base::paste(column_names_duplicated, collapse=", "), ".")
    stop()
}

pop_names_all <- frequency_files_info$column_name %>% .[! . %in% c("CHR", "POS", "ALT", "REF")] %>% base::strsplit(split="~") %>% base::sapply(`[`, 1) %>% base::unique()

In [5]:
pop_info <- list()
columns_selected <- c()

for (pop in pop_names_all)
{
    frac2_name <- paste0(pop, c("~num", "~deno"))
    if (all(frac2_name %in% frequency_files_info$column_name))
    {
        deno <- frequency_files_info %>% dplyr::filter(column_name %in% frac2_name) %>% .$denominator %>% base::unique()
        if (base::length(deno) > 1)
        {
            message("Population ", pop, " doesn't have the same total count in its numerator and denominator.")
            next
        }
        if (is.na(deno))
        {
            message("Population ", pop, " doesn't have the total count in its numerator or denominator.")
            next
        }
        pop_info[[pop]] <- list(data_type = "frac2", total = deno, lth = NA)
        columns_selected <- columns_selected %>% append(frac2_name)
        next
    }
    
    if (pop %in% frequency_files_info$column_name)
    {
        #
        deno <- frequency_files_info %>% dplyr::filter(column_name == pop) %>% .$denominator
        pop_info[[pop]] <- if (is.na(deno) || deno == 1) list(data_type = "freq", total = 1, lth = NA) else list(data_type = "frac1", total = deno, lth = NA)
        columns_selected <- columns_selected %>% append(pop)
        next
    }

    message("Population ", pop, " doesn't apply either fraction or frequency format.")
}

frequency_files_info_filtered <- frequency_files_info %>% filter(column_name %in% c("CHR", "POS", "ALT", "REF", columns_selected))

In [6]:
frequency_files_info_filtered

frequency_file,field,column_name,denominator
<chr>,<int>,<chr>,<dbl>
/mnt/archgen/public_data/HGDP+1KG_callset/stats/POS.all_filtered/PS.HGDP+1000G.txt,1,CHR,NA
/mnt/archgen/public_data/HGDP+1KG_callset/stats/POS.all_filtered/PS.HGDP+1000G.txt,2,POS,NA
/mnt/archgen/public_data/HGDP+1KG_callset/stats/POS.all_filtered/count/AFR_all.freq.txt,1,AFR_all,1
/mnt/archgen/public_data/HGDP+1KG_callset/stats/POS.all_filtered/count/CEU_norelated.frac.txt,1,CEU~num,242
/mnt/archgen/public_data/HGDP+1KG_callset/stats/POS.all_filtered/count/CEU_norelated.frac.txt,2,CEU~deno,242
/mnt/archgen/public_data/HGDP+1KG_callset/stats/POS.all_filtered/count/FIN_norelated.frac.txt,1,FIN~num,196
/mnt/archgen/public_data/HGDP+1KG_callset/stats/POS.all_filtered/count/FIN_norelated.frac.txt,2,FIN~deno,196
/mnt/archgen/public_data/HGDP+1KG_callset/stats/POS.all_filtered/count/GBR_norelated.frac.txt,1,GBR~num,176
/mnt/archgen/public_data/HGDP+1KG_callset/stats/POS.all_filtered/count/GBR_norelated.frac.txt,2,GBR~deno,176


In [7]:
pop_info[1:3]

$AFR_all
$AFR_all$data_type
[1] "freq"

$AFR_all$total
[1] 1

$AFR_all$lth
[1] NA


$CEU
$CEU$data_type
[1] "frac2"

$CEU$total
[1] 242

$CEU$lth
[1] NA


$FIN
$FIN$data_type
[1] "frac2"

$FIN$total
[1] 196

$FIN$lth
[1] NA

In [8]:
if (!is.na(pop_def_file))
{
    if (file.exists(pop_def_file))
    {
        pop_def_o <- utils::read.table(pop_def_file, header = FALSE, sep = "\t")[[1]] %>% base::strsplit(split=":") %>% .[! lapply(., `[`, 1) %in% names(pop_info)]
        pop_def <- pop_def_o %>% lapply(`[`, 2) %>% sapply(base::strsplit, split=",")
        for (i in length(pop_def))
            pop_def[[i]] <- pop_def[[i]] %>% .[. %in% names(pop_info)] # need to filter %in% names(pop_info)
        names(pop_def) <- pop_def_o %>% lapply(`[`, 1)
    } else
    {
        message("The population definition file ", pop_def_file , " does not exist.")
        pop_def_file <- NA
    }
        
}


In [9]:
pop_to_column <- function(pop)
{
    if (pop %in% names(pop_info))
    {
        data_type <- pop_info[[pop]]$data_type
        if (data_type == "frac2")
        {
            return(paste0(pop, c("~num", "~deno")))
        } else
        {
            return(pop)
        }
    } else if (pop %in% names(pop_def))
    {
        return(pop_def[[pop]] %>% lapply(pop_to_column) %>% unlist())
    }
}

In [10]:
column_name_full_length <- c(asc_refpop, asc_outgroup) %>% lapply(pop_to_column) %>% unlist()

In [11]:
column_name_full_length

[1] "CEU~num"  "CEU~deno" "FIN~num"  "FIN~deno" "GBR~num"  "GBR~deno"
 [7] "IBS~num"  "IBS~deno" "TSI~num"  "TSI~deno" "AFR_all"

In [12]:
pop_to_freq <- function(pop_all, mask = NA)
{
    pop_all_file_info <- frequency_files_info_filtered %>% dplyr::filter(column_name %in% pop_all)

    file_all <- pop_all_file_info$frequency_file %>% base::unique()

    geno_all <- list()
    for (file in file_all)
    {
        pop_file_info <- pop_all_file_info %>% filter(frequency_file == file)
        field <- pop_file_info$field
        pop <- pop_file_info$column_name

        is_geno <- tools::file_ext(file) == "geno"
        geno <- if (is_geno) readr::read_fwf(file, col_positions=readr::fwf_positions(field, field, col_names=pop), col_types=readr::cols(.default="i"), na="9") else data.table::fread(file=file, header=FALSE, select=field, na.strings=c("-1", "*"), col.names=pop)
        if (!is.na(mask))
            geno <- geno[mask, ]
        geno_all <- geno_all %>% append(geno)
    }

    return(geno_all)
}

In [13]:
column_full_length <- pop_to_freq(column_name_full_length)

In [35]:

get_freq <- function(pop, lth) # new parameters to add: keep_frac = FALSE
{
    #
    if (pop %in% names(pop_info))
    {
        if (!is.na(pop_info[[pop]]$lth))
            return()
        
        data_type <- pop_info[[pop]]$data_type
        if (data_type == "frac2")
        {
            pop_num <- paste0(pop, "~num")
            pop_deno <- paste0(pop, "~deno")
            
            pop_info[[pop]]$num <<- column_full_length[[pop_num]]
            pop_info[[pop]]$deno <<- column_full_length[[pop_deno]]
            pop_info[[pop]]$freq <<- ( column_full_length[[pop_num]] / column_full_length[[pop_deno]] ) %>% replace(is.nan(.), NA)
            pop_info[[pop]]$lth <<- lth
        } else if (data_type == "frac1")
        {
            geno <- column_full_length[[pop]]
            geno_na <- is.na(geno)
            
            pop_info[[pop]]$num <<- replace(geno, geno_na, 0)
            pop_info[[pop]]$deno <<- ifelse(geno_na, 0, pop_info[[pop]]$total)
            pop_info[[pop]]$freq <<- geno / pop_info[[pop]]$total
            pop_info[[pop]]$lth <<- lth
        } else if (data_type == "freq")
        {
            # message("Can not be added!")
            pop_info[[pop]]$freq <<- column_full_length[[pop]]
            pop_info[[pop]]$lth <<- lth
        }
    }
        
    else if (pop %in% names(pop_def))
    {
        num_sum <- 0
        deno_sum <- 0
        total_sum <- 0
        for (pop0 in pop_def[[pop]])
        {
            get_freq(pop0, lth)
            num_sum <- num_sum + pop_info[[pop0]]$num
            deno_sum <- deno_sum + pop_info[[pop0]]$deno
            total_sum <- total_sum + pop_info[[pop0]]$total
        }
        pop_info[[pop]] <<- list(data_type = "frac2", total = total_sum, num = num_sum, deno = deno_sum, freq = (num_sum / deno_sum) %>% replace(is.nan(.), NA), lth = lth)
    }
    
}

In [10]:
geno_all <- pop_to_freq(frequency_files_info_filtered$column_name)

In [9]:
names(pop_info)

[1] "AFR_all"           "CEU"               "FIN"              
 [4] "GBR"               "IBS"               "TSI"              
 [7] "HGDP01381"         "HGDP01382"         "HGDP01383"        
[10] "HGDP01384"         "HGDP01385"         "HGDP01386"        
[13] "HGDP01387"         "HGDP01388"         "HGDP01396"        
[16] "HGDP01397"         "HGDP01398"         "HGDP01399"        
[19] "HGDP01400"         "HGDP01403"         "HGDP01404"        
[22] "LP6005441-DNA_A01" "LP6005441-DNA_B01"

In [10]:
pop_def

$EUR5
[1] "CEU" "FIN" "GBR" "IBS" "TSI"

In [15]:
filter_outgroup_max_hetero <- pop_info[[asc_outgroup]]$freq <= outgroup_max_hetero | pop_info[[asc_outgroup]]$freq >= 1 - outgroup_max_hetero

filter_refpop_max_miss <- pop_info[[asc_refpop]]$deno >= pop_info[[asc_refpop]]$total * (1 - refpop_max_miss)

filter_refpop_remove_homo <- ! (pop_info[[asc_refpop]]$freq == 0 | pop_info[[asc_refpop]]$freq == 1)

In [16]:
filter_all <- ( if (refpop_remove_homo) filter_outgroup_max_hetero & filter_refpop_max_miss & filter_refpop_remove_homo else filter_outgroup_max_hetero & filter_refpop_max_miss ) %>% replace(is.na(.), FALSE)

In [17]:
for (pop in names(pop_info))
{
    #
    if (!is.null(pop_info[[pop]]$freq))
    {
        pop_info[[pop]]$num <- NULL
        pop_info[[pop]]$deno <- NULL
        pop_info[[pop]]$freq <- pop_info[[pop]]$freq[filter_all]
        next
    }

    data_type <- pop_info[[pop]]$data_type
    if (data_type == "frac2")
    {
        pop_num <- paste0(pop, "~num")
        pop_deno <- paste0(pop, "~deno")

        pop_info[[pop]]$freq <- ( geno_all[[pop_num]][filter_all] / geno_all[[pop_deno]][filter_all] ) %>% replace(is.nan(.), NA)
    } else if (data_type == "frac1")
    {
        pop_info[[pop]]$freq <- geno_all[[pop]][filter_all] / pop_info[[pop]]$total
    } else if (data_type == "freq")
    {
        pop_info[[pop]]$freq <- geno_all[[pop]][filter_all]
    }
}



 should modify outgroup case

if (!is.na(asc_freq_file))
{
    if (file.exists(asc_freq_file))
    {
        asc_freq <- utils::read.table(asc_freq_file, header = FALSE, sep = "\t")[[1]]
    } else
    {
        asc_freq_file <- NA
        message("The ascertainment file ", asc_freq_file , " does not exist. No ascertainment is defined.")
    }
}

if (is.na(asc_freq_file) && !is.na(outgroup))
{
    message("Outgroup only applies when ascertainment is defined.")
    outgroup <- NA
}

if (!is.na(outgroup) && ! outgroup %in% column_names_all)
{
    message("Outgroup ", outgroup, " is not in the population name list.")
    outgroup <- NA
}


In [18]:

pop_str_convert <- function(pop_str, universe)
{
    pop <- ( if (file.exists(pop_str)) utils::read.table(pop_str, header=FALSE, sep = "\t")[[1]] else base::strsplit(pop_str, ",")[[1]] ) %>% .[. %in% universe] %>% base::unique()
    message("Input ", length(pop), " valid population(s): ", base::paste(pop, collapse=", "), ".")
    return(pop)
}

message("PopA:")
popA_all <- pop_str_convert(popA_str, names(pop_info))
message("PopB:")
popB_all <- pop_str_convert(popB_str, names(pop_info))
if (n_pops >= 3)
{
    message("PopC:")
    popC_all <- pop_str_convert(popC_str, names(pop_info))
} else
    popC_all <- NA
if (n_pops == 4)
{
    message("PopD:")
    popD_all <- pop_str_convert(popD_str, names(pop_info))
} else
    popD_all <- NA



PopA:

Input 1 valid population(s): AFR_all.

PopB:

Input 5 valid population(s): CEU, FIN, GBR, IBS, TSI.

PopC:

Input 17 valid population(s): HGDP01381, HGDP01382, HGDP01383, HGDP01384, HGDP01385, HGDP01386, HGDP01387, HGDP01388, HGDP01396, HGDP01397, HGDP01398, HGDP01399, HGDP01400, HGDP01403, HGDP01404, LP6005441-DNA_A01, LP6005441-DNA_B01.



In [5]:
fill_na_with_ref <- function(tgt, ref)
{
    pos <- is.na(tgt)
    tgt[pos] <- if (length(ref)==1) ref else ref[pos]
    return(tgt)
}

In [7]:
if (!is.na(position_file) && !file.exists(position_file))
{
    position_file <- NA
    message("The position file", position_file, "does not exist.")
}
    
position <- if (!is.na(position_file)) utils::read.table(position_file, header = FALSE, sep = "\t", col.names = c("CHR", "POS")) else pop_to_freq(c("CHR", "POS"), frequency_files_info)

In [8]:
outgroup_freq <- pop_to_freq(outgroup, frequency_files_info)[[outgroup]]

In [9]:
popA_freq <- pop_to_freq(popA_all, frequency_files_info) %>% {if (!is.na(popA_fillna)) base::lapply(., fill_na_with_ref, ref=if (popA_fillna=="Outgroup") outgroup_freq else 0) else .}

In [10]:
popB_freq <- pop_to_freq(popB_all, frequency_files_info) %>% {if (!is.na(popB_fillna)) base::lapply(., fill_na_with_ref, ref=if (popB_fillna=="Outgroup") outgroup_freq else 0) else .}

In [11]:
if (n_pops >= 3)
{
    popC_freq <- pop_to_freq(popC_all, frequency_files_info) %>% {if (!is.na(popC_fillna)) base::lapply(., fill_na_with_ref, ref=if (popC_fillna=="Outgroup") outgroup_freq else 0) else .}
} else
    popC_freq <- NA

In [12]:
if (n_pops == 4)
{
    popD_freq <- pop_to_freq(popD_all, frequency_files_info) %>% {if (!is.na(popD_fillna)) base::lapply(., fill_na_with_ref, ref=if (popD_fillna=="Outgroup") outgroup_freq else 0) else .}
} else
    popD_freq <- NA

In [13]:
popA_all
popB_all
popC_all
popD_all

[1] "AFR_all"

[1] "CEU" "FIN" "GBR" "IBS" "TSI"

[1] "HGDP01381"         "HGDP01382"         "HGDP01383"        
 [4] "HGDP01384"         "HGDP01385"         "HGDP01386"        
 [7] "HGDP01387"         "HGDP01388"         "HGDP01396"        
[10] "HGDP01397"         "HGDP01398"         "HGDP01399"        
[13] "HGDP01400"         "HGDP01403"         "HGDP01404"        
[16] "LP6005441-DNA_A01" "LP6005441-DNA_B01"

[1] NA

In [15]:
all_stats <- tidyr::crossing(popA = popA_all, popB = popB_all, popC = popC_all, popD = popD_all)

In [15]:
block_id <- if (jackknife=="CHR") position[["CHR"]] else position[["CHR"]] * 100 + position[["POS"]] %/% jackknife

In [16]:
f_value <- switch(as.character(n_pops), "2"=(popB_freq[[popB]] - popA_freq[[popA]]) ^ 2, "3"=(popB_freq[[popB]] - popA_freq[[popA]]) * (popC_freq[[popC]] - popA_freq[[popA]]), "4"=(popB_freq[[popB]] - popA_freq[[popA]]) * (popD_freq[[popD]] - popC_freq[[popC]]))

In [62]:
# ascertainment
fsum_block_asc_cutoff <- data.table::data.table(block_id=block_id, asc_freq=asc_freq, f_value=f_value)[, .(Nr_sites_all=.N, Nr_sites_nonmissing=sum(!is.na(f_value)), fsum=sum(f_value, na.rm=TRUE)), .(block_id, asc_freq)][base::order(block_id, asc_freq)][, .(asc_freq_max=asc_freq, Nr_sites_all=cumsum(Nr_sites_all), Nr_sites_nonmissing=cumsum(Nr_sites_nonmissing), fsum=cumsum(fsum)), .(block_id)]

In [64]:
fsum_block_asc_cutoff[, c("popA", "popB", "popC", "popD") := data.frame(popA, popB, popC, popD)]

In [ ]:
# no ascertainment
fsum_block_asc_cutoff <- data.table::data.table(block_id=block_id, f_value=f_value)[, .(Nr_sites_all=.N, Nr_sites_nonmissing=sum(!is.na(f_value)), fsum=sum(f_value, na.rm=TRUE)), .(block_id)][base::order(block_id)]

In [89]:
get_blockfsum <- function(popA, popB, popC, popD)
{
    f_value <- switch(as.character(n_pops), "2"=(popB_freq[[popB]] - popA_freq[[popA]]) ^ 2, "3"=(popB_freq[[popB]] - popA_freq[[popA]]) * (popC_freq[[popC]] - popA_freq[[popA]]), "4"=(popB_freq[[popB]] - popA_freq[[popA]]) * (popD_freq[[popD]] - popC_freq[[popC]]))

    #return(f_value)

    if (!is.na(asc_freq_file))
    {
        fsum_block_asc_cutoff <- data.table::data.table(block_id=block_id, asc_freq=asc_freq, f_value=f_value)[, .(Nr_sites_all=.N, Nr_sites_nonmissing=sum(!is.na(f_value)), fsum=sum(f_value, na.rm=TRUE)), .(block_id, asc_freq)][base::order(block_id, asc_freq)][, .(asc_freq_max=asc_freq, Nr_sites_all=cumsum(Nr_sites_all), Nr_sites_nonmissing=cumsum(Nr_sites_nonmissing), fsum=cumsum(fsum)), .(block_id)]
    } else
        fsum_block_asc_cutoff <- data.table::data.table(block_id=block_id, f_value=f_value)[, .(Nr_sites_all=.N, Nr_sites_nonmissing=sum(!is.na(f_value)), fsum=sum(f_value, na.rm=TRUE)), .(block_id)][base::order(block_id)]
    
    return(fsum_block_asc_cutoff[, c("popA", "popB", "popC", "popD") := data.frame(popA, popB, popC, popD)])
}

In [98]:
fsum_block_asc_cutoff_all <- purrr::pmap(all_stats, get_blockfsum) %>% do.call(base::rbind, .)

In [91]:
library(furrr)

Loading required package: future



In [93]:
future::plan(multisession, workers = 8)

In [102]:
future_map(rep(2,16), ~Sys.sleep(.x))

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

In [96]:
options(future.globals.maxSize = 3000 * 1024^2)

In [ ]:
furrr::future_pmap(all_stats, get_blockfsum)# %>% do.call(base::rbind, .)

In [2]:
# input parameters

sfs_file <- "sfs_1Test/CHR.FIN.EUR_all.OAI009.sfs"

block_col <- 1

block_id_all <- 1:22

n_pops <- 3

pop_cols <- c(0,2,4)

pop_sizes <- c(NA,198,2)

count_col <- 5

asc_col <- 3

min_freq_all <- 1

max_freq_all <- 1:100

all_sites_block <- TRUE

min_sites_block <- 1000


In [2]:
# sfs2blockfsum: input parameters

sfs_file <- "/mnt/archgen/users/lei_huang/ancient_British/POS.AFR_mono.13EUR+NL+SE.EUR_asc_25p/f3.Out.EUR_pop.British_ind/sfs.d3.count250_round/CHR.NL.count250_round.12880A.freqSum"

block_col <- 1

block_id_all <- 1:22

n_pops <- 3

pop_cols <- c(0,2,4)

fill_homo_cols <- c(2) # new arguments to add

pop_sizes <- c(NA,1,2)

count_col <- NA

asc_col <- 3


In [3]:
# should consider the case when we don't have a specific frequency.

In [3]:
sfs <- utils::read.table(sfs_file, header = FALSE, sep = "\t", na.strings = c("-1", "*"))

In [4]:
if (!is.na(fill_homo_cols))
    sfs[is.na(sfs[, fill_homo_cols]), fill_homo_cols] <- 0

In [5]:
allele_freq <- list()
for (i in 1:n_pops)
   allele_freq[[i]] <- if (pop_cols[i]) sfs[, pop_cols[i]] / pop_sizes[i] else 0

# Compute f value (f2/f3/f4). For sites that are not available in all pops, the value would be NA.
sfs$f <- switch(as.character(n_pops), "2"=(allele_freq[[2]] - allele_freq[[1]]) ^ 2, "3"=(allele_freq[[2]] - allele_freq[[1]]) * (allele_freq[[3]] - allele_freq[[1]]), "4"=(allele_freq[[2]] - allele_freq[[1]]) * (allele_freq[[4]] - allele_freq[[3]]))

In [6]:
fsum_block_asc_cutoff <- data.frame()

In [7]:
# !is.na(asc_col)

asc_freq_all <- base::unique(sfs[, asc_col]) %>% base::sort()

#cumulative sum for cutoff min_freq_min:asc_freq_max, with asc_freq_max ranging from min_freq_min to max_freq_max
for (block_id in block_id_all)
{
    sfs_block <- sfs[sfs[, block_col]==block_id, ]

    Nr_sites_all <- 0
    Nr_sites_nonmissing <- 0
    fsum <- 0

    for (asc_freq_max in asc_freq_all)
    {
        sfs_block_asc <- sfs_block[sfs_block[, asc_col]==asc_freq_max, ]

        if (!is.na(count_col))
        {
            Nr_sites_all <- Nr_sites_all + sum(sfs_block_asc[, count_col])
            Nr_sites_nonmissing <- Nr_sites_nonmissing + sum(sfs_block_asc[!is.na(sfs_block_asc$f), count_col])
            fsum <- fsum + sum(sfs_block_asc[, count_col] * sfs_block_asc$f, na.rm = TRUE)
        } else
        {
            Nr_sites_all <- Nr_sites_all + nrow(sfs_block_asc)
            Nr_sites_nonmissing <- Nr_sites_nonmissing + sum(!is.na(sfs_block_asc$f))
            fsum <- fsum + sum(sfs_block_asc$f, na.rm = TRUE)
        }
        

        fsum_block_asc_cutoff <- base::rbind(fsum_block_asc_cutoff, list(block_id=block_id, asc_freq_max=asc_freq_max, Nr_sites_all=Nr_sites_all, Nr_sites_nonmissing=Nr_sites_nonmissing, fsum=fsum))
    }
}

In [9]:
sfs %>% dplyr::group_by(block_id = c_across(all_of(block_col)), asc_freq = c_across(all_of(asc_col))) %>% {if (is.na(count_col)) dplyr::summarise(., Nr_sites_all=dplyr::n(), Nr_sites_nonmissing=sum(!is.na(f)), fsum=sum(f, na.rm = TRUE)) else dplyr::summarise(., Nr_sites_all=sum(c_across(all_of(count_col))), Nr_sites_nonmissing=sum((!is.na(f)) * c_across(all_of(count_col))), fsum=sum(f * c_across(all_of(count_col)), na.rm = TRUE))} %>% dplyr::reframe(asc_freq_max=asc_freq, Nr_sites_all=cumsum(Nr_sites_all), Nr_sites_nonmissing=cumsum(Nr_sites_nonmissing), fsum=cumsum(fsum))

`summarise()` has grouped output by 'block_id'. You can override using the
`.groups` argument.


block_id,asc_freq_max,Nr_sites_all,Nr_sites_nonmissing,fsum
<int>,<int>,<int>,<int>,<dbl>
1,0,2316086,293700,0.06513011
1,1,2577582,324632,0.22945873
1,2,2628512,330347,0.49398769
1,3,2649953,332667,0.73346662
1,4,2662040,333924,0.90581142
1,5,2670048,334730,1.11122252
1,6,2675386,335231,1.35370752
1,7,2679327,335593,1.53206412
1,8,2682136,335813,1.60921842


is.na(asc_col)

for (block_id in block_id_all)
{
    sfs_block <- sfs[sfs[, block_col]==block_id, ]

    if (!is.na(count_col))
    {
        Nr_sites_all <- sum(sfs_block[, count_col])
        Nr_sites_nonmissing <- sum(sfs_block[!is.na(sfs_block_asc$f), count_col])
        fsum <- sum(sfs_block[, count_col] * sfs_block$f, na.rm = TRUE)
    } else
    {
        Nr_sites_all <- nrow(sfs_block)
        Nr_sites_nonmissing <- sum(!is.na(sfs_block$f))
        fsum <- sum(sfs_block$f, na.rm = TRUE)
    }

    fsum_block_asc_cutoff <- base::rbind(fsum_block_asc_cutoff, list(block_id=block_id, Nr_sites_all=Nr_sites_all, Nr_sites_nonmissing=Nr_sites_nonmissing, fsum=fsum))
}

In [68]:
# use "all_of" to select by external variable
# https://tidyselect.r-lib.org/reference/faq-external-vector.html

# %>% group_vars()
# format(object.size(sfs_block_asc), units="GB")

In [9]:
# Conditional pipelines
if (is.na(asc_col))
{
    fsum_block_asc_cutoff <- sfs %>% dplyr::group_by(block_id = c_across(all_of(block_col))) %>% {if (is.na(count_col)) dplyr::summarise(., Nr_sites_all=dplyr::n(), Nr_sites_nonmissing=sum(!is.na(f)), fsum=sum(f, na.rm = TRUE)) else dplyr::summarise(., Nr_sites_all=sum(c_across(all_of(count_col))), Nr_sites_nonmissing=sum((!is.na(f)) * c_across(all_of(count_col))), fsum=sum(f * c_across(all_of(count_col)), na.rm = TRUE))}
}

In [24]:
provided <- function(data, condition, call)
{
    if (rlang::eval_tidy(rlang::enquo(condition), data)) rlang::eval_tidy(rlang::quo_squash(rlang::quo(data %>% !!rlang::enquo(call)))) else data
}

In [26]:
if (is.na(asc_col))
{
    fsum_block_asc_cutoff <- sfs %>% dplyr::group_by(block_id = c_across(all_of(block_col))) %>% provided(is.na(count_col), dplyr::summarise(Nr_sites_all=dplyr::n(), Nr_sites_nonmissing=sum(!is.na(f)), fsum=sum(f, na.rm = TRUE))) %>% provided(!is.na(count_col), dplyr::summarise(Nr_sites_all=sum(c_across(all_of(count_col))), Nr_sites_nonmissing=sum((!is.na(f)) * c_across(all_of(count_col))), fsum=sum(f * c_across(all_of(count_col)), na.rm = TRUE)))
}

In [83]:
fsum_block_asc_cutoff

block_id,Nr_sites_all,Nr_sites_nonmissing,fsum
<int>,<int>,<int>,<dbl>
1,2722467,338384,27.971941
2,2892819,367698,7.438877
3,2353146,299926,9.979959
4,2311335,279032,5.321643
5,2135035,270604,4.085171
6,1989200,255080,14.951904
7,1952358,235818,19.601203
8,1846440,239749,21.247497
9,1549549,179329,8.943885
